In [ ]:
# 手順1
import pandas as pd
import numpy as np
import glob
import os

# 指定フォルダ内のすべてのtxtファイルを取得
txt_files = glob.glob(os.path.join(r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_txt_to_csv", "sms-call-internet-mi-*.txt"))

# 列名を定義
columns = ["CellID", "datetime", "countrycode", "smsin", "smsout", "callin", "callout", "internet"]

# 各txtファイルを処理してcsvに変換
for txt_file_path in txt_files:
    csv_file_path = txt_file_path.replace(".txt", ".csv")
    
    # 既にcsvファイルが存在する場合はスキップ
    if not os.path.exists(csv_file_path):
        dataset = pd.read_csv(txt_file_path, delimiter="\t", names=columns)
        dataset.to_csv(csv_file_path, index=False)
        print(f"Converted {os.path.basename(txt_file_path)} to {os.path.basename(csv_file_path)}")
    else:
        print(f"Skipping {os.path.basename(txt_file_path)}, already converted.")


In [ ]:
### 手順２ （Datasets_txt_to_csv内に、mi-hour-*, mi-minute-*を作成）
import pandas as pd
import numpy as np
import glob
import os

##  csvファイルを処理してデータを整形
def format_data(time_unit: str):
    # 指定ディレクトリ内のcsvファイルを取得
    csv_files = glob.glob(os.path.join(r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_txt_to_csv", "sms-call-internet-mi-*.csv"))

    # 列名を定義
    columns = ["datetime", "CellID", "countrycode", "smsin", "smsout", "callin", "callout", "internet"]
    columns_order = ['datetime', 'CellID', 'sms', 'call', 'internet']

    # 各列の型変換を辞書に格納
    conversion_dict = {
        'datetime': 'datetime64[ns]',
        'CellID': 'int16',
        'smsin': 'float64',
        'smsout': 'float64',
        'callin': 'float64',
        'callout': 'float64',
        'internet': 'float64',
    }

    # 各csvファイルを処理して列順を変換（smsとcallをまとめる）
    for csv_file_path in csv_files:
        dataset = pd.read_csv(csv_file_path)
        dataset = dataset[columns]
        dataset['datetime'] = pd.to_datetime(dataset['datetime'], unit='ms')
        dataset = dataset.sort_values(by=["CellID", "datetime"]).reset_index(drop=True)

        dataset.fillna(0, inplace=True)
        dataset = dataset.astype(conversion_dict)
        dataset['sms'] = dataset['smsin'] + dataset['smsout']
        dataset['call'] = dataset['callin'] + dataset['callout']
        dataset = dataset[columns_order]
        dataset = dataset[['datetime', 'CellID', 'sms', 'call', 'internet']].groupby(['datetime', 'CellID'], as_index=False).sum()

        if time_unit == 'minute':
            output_file_path = os.path.join(os.path.dirname(csv_file_path), f"mi-minute-{dataset['datetime'].max().strftime('%Y-%m-%d')}.csv")
        elif time_unit == 'hour':
            dataset['datetime'] = dataset['datetime'].dt.floor('1H')
            dataset = dataset.groupby(['datetime', 'CellID']).sum().reset_index()
            output_file_path = os.path.join(os.path.dirname(csv_file_path), f"mi-hour-{dataset['datetime'].max().strftime('%Y-%m-%d')}.csv")
        else:
            raise ValueError("time_unit must be 'minute' or 'hour'")
        
        dataset.to_csv(output_file_path, index=False)
        print(f"Processed file: {os.path.basename(csv_file_path)} -> {os.path.basename(output_file_path)}")

# 呼び出し例：
format_data('minute')
format_data('hour')

In [ ]:
# 手順３（Datasets_IDディレクトリ内に、指定した範囲のグリッドデータを出力）
import pandas as pd
import numpy as np
import glob
import os
import get_cellid_within_range

def filter_and_save_data(time_unit):
    # ファイルを取得
    csv_files = glob.glob(os.path.join(r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_txt_to_csv", f"mi-{time_unit}*.csv"))
    
    # 出力ディレクトリを設定
    if time_unit == 'hour':
        output_dir = r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_ID\hour"
    elif time_unit == 'minute':
        output_dir = r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_ID\minute"
    else:
        raise ValueError("time_unit must be 'minute' or 'hour'")
    
    # ディレクトリが存在しない場合は作成
    os.makedirs(output_dir, exist_ok=True)
    
    # 指定した範囲のIDを取得（中心となる'CellID', 一辺の長さ）
    cells_in_range = get_cellid_within_range.get_cells_in_range(5161, 30)

    # セルIDごとのデータを格納する辞書を初期化
    cell_filtered_datasets = {cell_id: [] for cell_id in cells_in_range}

    # 各csvファイルを処理
    for csv_file_path in csv_files:
        dataset = pd.read_csv(csv_file_path)
        dataset['datetime'] = pd.to_datetime(dataset['datetime'])

        # 各セルIDに対するデータをフィルタリング
        for cell_id in cells_in_range:
            cell_data = dataset[dataset['CellID'] == cell_id]
            if not cell_data.empty:  # データがある場合のみ追加
                cell_filtered_datasets[cell_id].append(cell_data)

    # すべてのCSVファイルを処理した後にデータを保存
    for cell_id, cell_data_list in cell_filtered_datasets.items():
        if cell_data_list:  # データがある場合のみ処理
            combined_cell_dataset = pd.concat(cell_data_list, ignore_index=True)
            
            # 出力ファイルパスを作成
            cell_output_file_path = os.path.join(output_dir, f"mi-ID-{cell_id}.csv")
            combined_cell_dataset.to_csv(cell_output_file_path, index=False)
            
            print(f"cell_id {cell_id} のデータを保存しました: {cell_output_file_path}")


filter_and_save_data('hour')
filter_and_save_data('minute')

In [ ]:
# 手順４　（Dataset_ID内の欠損データの補完）
import pandas as pd
import os
import glob

def fill_missing_data(time_unit):
    
    # ディレクトリの設定
    if time_unit == 'hour':
        base_dir = r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_ID\hour"
    elif time_unit == 'minute':
        base_dir = r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_ID\minute"
    else:
        raise ValueError("time_unitは 'hour' または 'minute' のいずれかを指定してください。")

    # base_file_pathは固定のmi-ID-5161.csv
    base_file_path = os.path.join(base_dir, "mi-ID-5161.csv")
    
    # base_file_pathの読み込み
    base_time = pd.read_csv(base_file_path, parse_dates=['datetime'])

    # mi-ID-*.csv形式のファイルを取得
    target_files = glob.glob(os.path.join(base_dir, "mi-ID-*.csv"))

    # mi-ID-5161.csvを除外（base_timeとして使うファイルは除外する）
    target_files = [file for file in target_files if file != base_file_path]

    # 各ターゲットファイルに対して処理を行う
    for target_file_path in target_files:
        print(f"処理中: {target_file_path}")
        
        # ターゲットファイルを読み込む
        target = pd.read_csv(target_file_path, parse_dates=['datetime'])

        # base_timeのdatetime列を基準にインデックスを作成
        base_time_index = base_time['datetime']

        # ファイル2をdatetime列でインデックス化し、基準のインデックスに再インデックス
        target = target.set_index('datetime').reindex(base_time_index)

        # 欠損値補完
        # CellIDは同じ値を維持
        target['CellID'] = target['CellID'].fillna(method='ffill').fillna(method='bfill')

        # sms, call, internetは0で補完
        target[['sms', 'call', 'internet']] = target[['sms', 'call', 'internet']].fillna(0)

        # インデックスをリセットして元の形式に戻す
        target = target.reset_index()

        # 補完後のデータを元のファイルに上書き保存
        target.to_csv(target_file_path, index=False)

        # 結果の確認（先頭5行を表示）
        print(target.head())
        print(f"補完後のデータを {target_file_path} に上書き保存しました。\n")

# 使用例
fill_missing_data('hour')  # 'hour'を指定した場合
fill_missing_data('minute')  # 'minute'を指定した場合

In [ ]:
## 手順５（Datasets_hour, Datasets_minute内に、train_*, test_*を作成）
import pandas as pd
import numpy as np
import glob
import os

## 11月分の訓練データと12月の第1週分のテストデータを取得するコード
def process_and_split_data(time_unit):
    # time_unitに基づいてファイルパスを取得
    if time_unit == 'minute':
        file_paths = glob.glob(os.path.join(r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_ID", "minute", "mi-ID-*.csv"))
        output_train = r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_minute\train"
        output_test = r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_minute\test"
    elif time_unit == 'hour':
        file_paths = glob.glob(os.path.join(r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_ID", "hour", "mi-ID-*.csv"))
        output_train = r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_hour\train"
        output_test = r"C:\Users\goshima\Documents\卒業研究_AI\Python\Datasets\Datasets_hour\test"
    else:
        print(f"Invalid time_unit: {time_unit}. Use 'minute' or 'hour'.")
        return
    
    # 出力先ディレクトリが存在しない場合は作成
    os.makedirs(output_train, exist_ok=True)
    os.makedirs(output_test, exist_ok=True)

    # 各ファイルに対して処理を実行
    for file_path in file_paths:
        dataset = pd.read_csv(file_path)
        dataset['datetime'] = pd.to_datetime(dataset['datetime'])

        # 10月と12月のデータを除外（11月のデータ）
        train_dataset = dataset[(dataset['datetime'].dt.month != 10) & (dataset['datetime'].dt.month != 12)]

        # 12/1 0:00:00 ~ 12/8 23:59:59 間のデータを取得
        test_dataset = dataset[(dataset['datetime'] >= '2013-12-01') & (dataset['datetime'] < '2013-12-09')]

        # 出力ファイル名を動的に生成
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        train_file = f"train_{base_name}.csv"
        test_file = f"test_{base_name}.csv"

        # ファイルパスにディレクトリを追加
        train_file_path = os.path.join(output_train, train_file)
        test_file_path = os.path.join(output_test, test_file)

        # 結果を保存
        train_dataset.to_csv(train_file_path, index=False)
        test_dataset.to_csv(test_file_path, index=False)

        print(f"Processed file: {os.path.basename(file_path)} -> {train_file}, {test_file}")

# 関数の使用例
process_and_split_data('hour')
process_and_split_data('minute')